# Rotator Night Observing integration test
This notebook is related to the JIRA test case: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T1593

In [2]:
import logging
import yaml

import os
import sqlite3
import pandas as pd

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio
from lsst.ts import salobj
from lsst.ts.idl.enums import MTPtg

from astropy.utils import iers
iers.conf.auto_download = False

from datetime import datetime

In [3]:
test_message = "Rotator Night Observing integration test - LVV-T1593"

The next cell opens the database for query. 

In [4]:
con = sqlite3.connect(os.path.expanduser("~/develop/one_filt_v1.4_0yrs_1.db"))

OperationalError: unable to open database file

Once the database is opened/connected we can query for all the entries in the `SummaryAllProps` database, which contains the information about each target observed. 

In [5]:
df = pd.read_sql_query("SELECT * from SummaryAllProps", con)

NameError: name 'con' is not defined

In [6]:
d = salobj.Domain()

In [7]:
script = salobj.Controller("Script", index=1)
rot = salobj.Remote(d, "MTRotator")
mtptg = salobj.Remote(d, "MTPtg")
mtm = salobj.Remote(d, "MTMount")

In [8]:
await asyncio.gather(rot.start_task,
                     mtptg.start_task,
                     script.start_task,
                     mtm.start_task)

rotation DDS read queue is full (100 elements); data may be lost
motors DDS read queue is full (100 elements); data may be lost
electrical DDS read queue is full (100 elements); data may be lost
ccwFollowingError DDS read queue is full (100 elements); data may be lost


[None, None, None, None]

timeAndDate DDS read queue is filling: 12 of 100 elements
mountStatus DDS read queue is filling: 13 of 100 elements
mountPosition DDS read queue is filling: 14 of 100 elements
currentTargetStatus DDS read queue is filling: 14 of 100 elements


In [9]:
await asyncio.sleep(1.)
await salobj.set_summary_state(mtptg, salobj.State.ENABLED)

[<State.FAULT: 3>, <State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [ ]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

In [ ]:
await salobj.set_summary_state(mtm, salobj.State.ENABLED)

Start the loop on the different field positions df given by the scheduler

In [ ]:
### Read the scheduler output positions for the next 8 hours
print("Number of visits:", len(df))

In [ ]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

# Move to 0 deg Starting Position

In [ ]:
now = datetime.now()
print("Start from Park position", now)
script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"START - {test_message} - Move to 0 deg Starting Position - {now} UTC")

alt = 60. * u.deg
az = 0. * u.deg
rot_tel = Angle(0, unit= u.deg) 

target_name="Rotator test"
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_mtptg = Time(time_data.mjd, format="mjd", scale="tai")
time_err = curr_time_mtptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=alt, az=az, 
                 obstime=curr_time_mtptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)

# Calculating the other parameters     
rot_pa = rot_tel
print(rot_pa)  

await rot.cmd_trackStart.start(timeout=30.)

ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    frame=MTPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    trackId=9999,
    rotAngle=15.0,
    rotStartFrame=MTPtg.RotFrame.FIXED,
    rotTrackFrame=MTPtg.RotFrame.FIXED,
    rotMode=MTPtg.RotMode.FIELD,
    azWrapStrategy=2,
    timeOnTarget=30,
    timeout=10
)


print("Waiting 30s")
await asyncio.sleep(30.)

In [ ]:
await mtptg.cmd_stopTracking.start(timeout=10.)
await mtm.cmd_stop.start(timeout=10.)
await rot.cmd_stop.start(timeout=10.)
await asyncio.sleep(0.5)

# Start Test

In [ ]:
now = datetime.now()
print("Start Test", now)
script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"START - {test_message} - {now} UTC")

In [ ]:
## start the loop that take a sky position one at the time and point and track

#for field in range(len(df)): 
for field in range(3): 
    print(field)
    target_name = df["note"][field]
    alt = df['altitude'][field] * u.deg
    az = df['azimuth'][field] * u.deg
    rot_tel = Angle(df["rotTelPos"][field]*u.deg)
    para_ang = Angle(df["paraAngle"][field]*u.deg)
    rot_pa = para_ang-180*u.deg+rot_tel.wrap_at('180d')
    visit_time = df['visitTime'][field]  # how long the visit lasted. Usefull so you can wait between visits
    print(f"Alt: {alt}\nAz: {az}\nRotTel: {rot_tel}")

#Here is a small trick to get the ra before transit. Get `timeAndDate` from the pointing component, then use `RA = lst - delta`.
    time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
    curr_time_atptg = Time(time_data.mjd, format="mjd", scale="tai")

# Compute RA/Dec for commanded az/el
    cmd_elaz = AltAz(alt=alt, az=az, 
                 obstime=curr_time_atptg.tai, 
                 location=location)
    cmd_radec = cmd_elaz.transform_to(ICRS) # This is the RA/Dec of the target

    print(f"RA: {cmd_radec.ra}\nDec: {cmd_radec.dec}\nRotPA: {rot_pa}")
    
    await rot.cmd_trackStart.start(timeout=30.)
    
    ack = await mtptg.cmd_raDecTarget.set_start(
        targetName=target_name,
        frame=MTPtg.CoordFrame.ICRS,
        epoch=2000,  # should be ignored: no parallax or proper motion
        equinox=2000,  # should be ignored for ICRS
        ra=cmd_radec.ra.hour,
        declination=cmd_radec.dec.deg,
        parallax=0,
        pmRA=0,
        pmDec=0,
        rv=0,
        dRA=0,
        dDec=0,
        trackId=9999,
        rotAngle=15.0,
        rotStartFrame=MTPtg.RotFrame.FIXED,
        rotTrackFrame=MTPtg.RotFrame.TARGET,
        rotMode=MTPtg.RotMode.FIELD,
        azWrapStrategy=2,
        timeOnTarget=30,
        timeout=10
    )
    print("Waiting", visit_time, "s")
    await asyncio.sleep(visit_time)
    
    #Workaround for Rotator failing between targets
    await mtptg.cmd_stopTracking.start(timeout=10.)
    await mtm.cmd_stop.start(timeout=10.)
    await rot.cmd_stop.start(timeout=10.)
    await asyncio.sleep(0.5)
          
now = datetime.now()
print("This is the end of the observing run", now)

# Move to Park position 

In [ ]:
now = datetime.now()
print("Move to park position", now)
script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"START - {test_message} - Move to Park position - {now} UTC")

alt = 60. * u.deg
az = 0. * u.deg
rot_tel = Angle(0, unit= u.deg) 

target_name="Rotator test"
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_mtptg = Time(time_data.mjd, format="mjd", scale="tai")
time_err = curr_time_mtptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=alt, az=az, 
                 obstime=curr_time_mtptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)

# Calculating the other parameters     
rot_pa = rot_tel
print(rot_pa)

await rot.cmd_trackStart.start(timeout=30.)
    

ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    frame=MTPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    trackId=9999,
    rotAngle=15.0,
    rotStartFrame=MTPtg.RotFrame.FIXED,
    rotTrackFrame=MTPtg.RotFrame.FIXED,
    rotMode=MTPtg.RotMode.FIELD,
    azWrapStrategy=2,
    timeOnTarget=30,
    timeout=10
)

print("Waiting 1s")
await asyncio.sleep(1.)

In [ ]:
now = datetime.now()
print("Stopping tracking", now)

#we need to stop the pointing commands before stopping the rotator. This will be integrated in the pointing code
await mtptg.cmd_stopTracking.start(timeout=10.)
await mtm.cmd_stop.start(timeout=10.)
await rot.cmd_stop.start(timeout=10.)

# Test Complete

In [ ]:
now = datetime.now()
print("Test Complete",now)

script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"END - {test_message} - {now} UTC")

# Additional Error Handling Commands

# Bring the Rotator and CCW back to enabled state

In [ ]:
await mtm.cmd_clearerror.start(timeout=30)

Wait for override off

In [1]:
await rot.cmd_clearError.start(timeout=30)

NameError: name 'rot' is not defined

In [ ]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

In [ ]:
await mtm.cmd_enterControl.start(timeout=30.)
await mtm.cmd_start.start(timeout=30.)
await mtm.cmd_enable.start(timeout=30.)

# Move to sync position with CCW

In [ ]:
print("Move to X.X deg starting position")

await rot.cmd_positionSet.set_start(angle=19.04,timeout=30.)

await rot.cmd_move.start(timeout=30.)

In [10]:
await rot.cmd_clearError.start(timeout=10)

In [ ]:
await rot.cmd_exitControl.start(timeout=10.)

In [ ]:
await rot.cmd_enterControl.start(timeout=10.)

In [11]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [ ]:
await rot.cmd_stop.start(timeout=10.)

In [ ]:
await mtm.cmd_clearerror.start(timeout=10)

In [ ]:
await mtptg.cmd_stopTracking.start(timeout=10)
await mtm.cmd_stop.start(timeout=10)

In [ ]:
await mtm.cmd_stop.start(timeout=10)

Bring CCW Down

In [ ]:
await mtm.cmd_disable.start(timeout=30.)
await mtm.cmd_standby.start(timeout=30.)
await mtm.cmd_exitControl.start(timeout=30.)

Wait for override off

In [ ]:
await mtm.cmd_enterControl.start(timeout=30.)
await mtm.cmd_start.start(timeout=30.)
await mtm.cmd_enable.start(timeout=30.)

In [ ]:
await mtptg.cmd_stopTracking.start(timeout=30)

In [ ]:
await salobj.set_summary_state(mtptg, salobj.State.ENABLED)